## Transfer Learning Tuning_EyeWear:
  * model parameter and other information can be found below:
     * [source](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)
     
  * base_model: use vgg16 and freeze at bottleneck layer (stop right before flatten layer) 
  * top_model: tune dense layers (parameters are inspired by source)
     * batch_size 16 seems to work best for small data set 
     
  * extra blog source for callbacks and saving models:
  
  [call_backs for best weights](https://machinelearningmastery.com/check-point-deep-learning-models-keras/)
  
  [saving models](https://machinelearningmastery.com/save-load-keras-deep-learning-models/)

##### warnings: due to the order of my dataframe, the first entry is not_eyewear, class predict 1 == has no eyewear
  
---
#### This cell is required in order to use GPU for running the model 

In [1]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import keras
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras
keras.backend.get_session().run(tf.global_variables_initializer())  

Using TensorFlow backend.
W0831 22:13:27.989720 140695323678528 deprecation_wrapper.py:119] From /home/mindy/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0831 22:13:27.990682 140695323678528 deprecation_wrapper.py:119] From /home/mindy/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:207: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



---
#### Import necessary libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from keras_preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense
from keras.applications.vgg16 import VGG16, preprocess_input
from keras import optimizers
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint

---
#### Import train_df and test_df for eyewears

In [3]:
train_df = pd.read_pickle('../pickle_files/train_df_glasses.pkl')
test_df = pd.read_pickle('../pickle_files/test_df_glasses.pkl')

In [ ]:
train_df.head(2)

#### Get bottleneck features to tune top models

In [4]:
def save_bottleneck_features(train_df, test_df, label, batch_size):
    '''
    inputs:
    train_df, test_df: train and test dataframes saved in pickle_files folder
    label: a string, eyewear, hat, or beard
    batch_size: process images in batches
    outputs:
    saves bottleneck features inside folder tuning_data as npy file
    '''
    # intialize the vgg16 model 
    # make sure not to train the top layers 
    base_model = VGG16(weights = 'imagenet', include_top = False)
    # create train_generator and test_generator to get bottleneck inputs for train and test df 
    datagen = ImageDataGenerator(
        rescale=1./255,
        zoom_range= [0.8,1.7],
        shear_range=0.2,
        rotation_range = 40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        preprocessing_function=preprocess_input) # add reprocess to prevent overfitting 
    # make sure shuffle is False so we know the label follows the sequence of the dataframe 
    # so we can tune top_model 
    train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='../data/pics',
    x_col='pic_id',
    y_col=label,
    batch_size=batch_size,
    shuffle = False,
    target_size=(224,224),
    class_mode = None)
    # get features saved as .npy in tunign_data folder 
    bottleneck_features_train = base_model.predict_generator(
        train_generator, train_df.shape[0]//batch_size)
    np.save(open('../tuning_data/bottleneck_features_train_eyewears_3.npy','wb'),
           bottleneck_features_train)
    
    test_generator = datagen.flow_from_dataframe(
    dataframe=test_df,
    directory='../data/pics',
    x_col='pic_id',
    y_col=label,
    batch_size=batch_size,
    shuffle = False,
    target_size=(224,224),  # change to default input size 
    class_mode = None)
    bottleneck_features_test = base_model.predict_generator(
        test_generator, test_df.shape[0]//batch_size)
    np.save(open('../tuning_data/bottleneck_features_test_eyewears_3.npy','wb'),
           bottleneck_features_test)

#### Save bottleneck_features

In [5]:
save_bottleneck_features(train_df,test_df,'eyewear',16)

W0831 22:13:39.319429 140695323678528 deprecation_wrapper.py:119] From /home/mindy/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0831 22:13:39.320162 140695323678528 deprecation_wrapper.py:119] From /home/mindy/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0831 22:13:39.322047 140695323678528 deprecation_wrapper.py:119] From /home/mindy/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0831 22:13:39.340845 140695323678528 deprecation_wrapper.py:119] From /home/mindy/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4267: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



Found 505 validated image filenames.
Found 127 validated image filenames.


### Quick tuning of top models 

In [6]:
def train_top_model(train_df, test_df, epoch, batch_size, label):
    '''
    inputs:
    train_df, test_df: dataframes saved in pickle_files to generate train and test labels 
    epoch: num of epochs in fit 
    batch_size: same as image generator batch size 
    label: a string, eyewear, hat, or beard
    output:
    saves model weights in a folder 
    '''
    train_data = np.load(open('../tuning_data/bottleneck_features_train_eyewears_3.npy','rb'))
    # make sure train_data and train_label have same num of samples
    train_label = np.array(train_df[label].map({'not_'+label:0, label:1}))[:-(train_df.shape[0]%batch_size)]
    
    test_data = np.load(open('../tuning_data/bottleneck_features_test_eyewears_3.npy','rb'))
    test_label = np.array(test_df[label].map({'not_'+label:0, label:1}))[:-(test_df.shape[0]%batch_size)]
    
    # build top model
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile model 
    model.compile(optimizer='rmsprop',
                 loss='binary_crossentropy',
                 metrics=['accuracy'])
    
    # checkpoint for best weights 
    filepath="../tuning_data/best_bottleneck_vgg_model_eyewears_3.h5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    
    # fit model 
    model.fit(train_data, train_label,
             epochs=epoch,
             batch_size=batch_size,
             validation_data=(test_data,test_label),
             callbacks=callbacks_list)
    del model
    keras.backend.clear_session()

#### run train_top_model and save results in tuning_data

In [7]:
train_top_model(train_df, test_df, 50, 16, 'eyewear')

W0831 22:13:48.482006 140695323678528 deprecation.py:506] From /home/mindy/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3733: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0831 22:13:48.496432 140695323678528 deprecation_wrapper.py:119] From /home/mindy/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0831 22:13:48.507232 140695323678528 deprecation.py:323] From /home/mindy/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as n

Train on 496 samples, validate on 112 samples
Epoch 1/50
496/496 [==============================] - 1s 1ms/step - loss: 4.1279 - acc: 0.5383 - val_loss: 1.0766 - val_acc: 0.5893

Epoch 00001: val_acc improved from -inf to 0.58929, saving model to ../tuning_data/best_bottleneck_vgg_model_eyewears_3.h5
Epoch 2/50
496/496 [==============================] - 0s 278us/step - loss: 1.2536 - acc: 0.6331 - val_loss: 0.5363 - val_acc: 0.7321

Epoch 00002: val_acc improved from 0.58929 to 0.73214, saving model to ../tuning_data/best_bottleneck_vgg_model_eyewears_3.h5
Epoch 3/50
496/496 [==============================] - 0s 286us/step - loss: 0.7357 - acc: 0.7440 - val_loss: 0.3180 - val_acc: 0.8661

Epoch 00003: val_acc improved from 0.73214 to 0.86607, saving model to ../tuning_data/best_bottleneck_vgg_model_eyewears_3.h5
Epoch 4/50
496/496 [==============================] - 0s 281us/step - loss: 0.5202 - acc: 0.7863 - val_loss: 0.3577 - val_acc: 0.8393

Epoch 00004: val_acc did not improve from

496/496 [==============================] - 0s 283us/step - loss: 0.0539 - acc: 0.9859 - val_loss: 0.4655 - val_acc: 0.9107

Epoch 00042: val_acc did not improve from 0.92857
Epoch 43/50
496/496 [==============================] - 0s 283us/step - loss: 1.1601e-04 - acc: 1.0000 - val_loss: 0.4603 - val_acc: 0.9107

Epoch 00043: val_acc did not improve from 0.92857
Epoch 44/50
496/496 [==============================] - 0s 280us/step - loss: 0.0168 - acc: 0.9960 - val_loss: 0.7210 - val_acc: 0.8750

Epoch 00044: val_acc did not improve from 0.92857
Epoch 45/50
496/496 [==============================] - 0s 295us/step - loss: 0.0065 - acc: 0.9960 - val_loss: 0.5674 - val_acc: 0.9107

Epoch 00045: val_acc did not improve from 0.92857
Epoch 46/50
496/496 [==============================] - 0s 292us/step - loss: 0.0065 - acc: 0.9980 - val_loss: 0.5213 - val_acc: 0.8929

Epoch 00046: val_acc did not improve from 0.92857
Epoch 47/50
496/496 [==============================] - 0s 289us/step - loss: 0

### Fine Tune Top Model to improve accuracy

In [9]:
def fine_tune_model(train_df, test_df,epoch, batch_size,label, print_model = True):
    # build VGG16 model and freeze top layers
    # input_shape: width, height, RGB (from image generator)
    model_vgg = VGG16(weights='imagenet',include_top=False, input_shape=(224,224,3))
    # build top model
    top_model = Sequential()
    top_model.add(Flatten(input_shape=model_vgg.output_shape[1:]))
    top_model.add(Dense(256,activation='relu'))
    top_model.add(Dropout(0.5))
    top_model.add(Dense(1, activation='sigmoid'))
    
    # load saved weights to fine tune parameters 
    top_model.load_weights('../tuning_data/best_bottleneck_vgg_model_eyewears_3.h5')
    # add top model to model
    model = Model(inputs=model_vgg.input, outputs=top_model(model_vgg.output))
    # we will tune last 5 layers of the model: block5 and fully connected layer 
    for layer in model.layers[:15]:
        layer.trainable = False
    # we can tune the parameters for lr and momentum later to get better results
    model.compile(loss='binary_crossentropy',
             optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
             metrics=['accuracy'])
    # prepare train generator using data augmentation to battle small sample size 
    train_gen = ImageDataGenerator(
        rescale=1./255,
        zoom_range= [0.8,1.7],
        shear_range=0.2,
        rotation_range = 40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        preprocessing_function=preprocess_input)
    # not want to augment the test 
    test_gen = ImageDataGenerator(
            rescale = 1./255,
            preprocessing_function=preprocess_input)
    
    train_generator =  train_gen.flow_from_dataframe(
    dataframe=train_df,
    directory='../data/pics',
    x_col='pic_id',
    y_col=label,
    batch_size=batch_size,
    target_size=(224,224),
    class_mode = 'binary')
    
    test_generator =  test_gen.flow_from_dataframe(
    dataframe=test_df,
    directory='../data/pics',
    x_col='pic_id',
    y_col=label,
    batch_size=batch_size,
    target_size=(224,224),
    class_mode = 'binary')
    
    # checkpoint for best weights 
    filepath="../tuning_data/best_vgg_model_eyewears_3.h5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    
    # run and fit model 
    result = model.fit_generator(
    train_generator,
    steps_per_epoch=train_df.shape[0]//batch_size,
    epochs=epoch,
    validation_data=test_generator,
    validation_steps=test_df.shape[0]//batch_size,
    verbose=1,
    callbacks=callbacks_list)
    
    if print_model:
        model.summary()
        
    del model
    keras.backend.clear_session()
    
    return result      

### save model history 

In [10]:
model_history = fine_tune_model(train_df, test_df,100,16,'eyewear')

Found 505 validated image filenames belonging to 2 classes.
Found 127 validated image filenames belonging to 2 classes.
Epoch 1/100
31/31 [==============================] - 7s 221ms/step - loss: 2.0686 - acc: 0.3121 - val_loss: 0.7907 - val_acc: 0.4286

Epoch 00001: val_acc improved from -inf to 0.42857, saving model to ../tuning_data/best_vgg_model_eyewears_3.h5
Epoch 2/100
31/31 [==============================] - 7s 219ms/step - loss: 0.7594 - acc: 0.5058 - val_loss: 0.7204 - val_acc: 0.4505

Epoch 00002: val_acc improved from 0.42857 to 0.45045, saving model to ../tuning_data/best_vgg_model_eyewears_3.h5
Epoch 3/100
31/31 [==============================] - 6s 198ms/step - loss: 0.7304 - acc: 0.4781 - val_loss: 0.7036 - val_acc: 0.4775

Epoch 00003: val_acc improved from 0.45045 to 0.47748, saving model to ../tuning_data/best_vgg_model_eyewears_3.h5
Epoch 4/100
31/31 [==============================] - 7s 213ms/step - loss: 0.7116 - acc: 0.4841 - val_loss: 0.6886 - val_acc: 0.5225

Ep

31/31 [==============================] - 6s 197ms/step - loss: 0.1344 - acc: 0.9456 - val_loss: 0.1402 - val_acc: 0.9730

Epoch 00039: val_acc did not improve from 0.99099
Epoch 40/100
31/31 [==============================] - 7s 211ms/step - loss: 0.1519 - acc: 0.9536 - val_loss: 0.0967 - val_acc: 0.9820

Epoch 00040: val_acc did not improve from 0.99099
Epoch 41/100
31/31 [==============================] - 6s 201ms/step - loss: 0.1346 - acc: 0.9445 - val_loss: 0.1428 - val_acc: 0.9554

Epoch 00041: val_acc did not improve from 0.99099
Epoch 42/100
31/31 [==============================] - 7s 229ms/step - loss: 0.1139 - acc: 0.9556 - val_loss: 0.0569 - val_acc: 0.9910

Epoch 00042: val_acc improved from 0.99099 to 0.99099, saving model to ../tuning_data/best_vgg_model_eyewears_3.h5
Epoch 43/100
31/31 [==============================] - 6s 204ms/step - loss: 0.1240 - acc: 0.9496 - val_loss: 0.1141 - val_acc: 0.9730

Epoch 00043: val_acc did not improve from 0.99099
Epoch 44/100
31/31 [===

31/31 [==============================] - 7s 222ms/step - loss: 0.0407 - acc: 0.9839 - val_loss: 0.1085 - val_acc: 0.9820

Epoch 00082: val_acc did not improve from 1.00000
Epoch 83/100
31/31 [==============================] - 7s 212ms/step - loss: 0.0435 - acc: 0.9859 - val_loss: 0.0879 - val_acc: 0.9910

Epoch 00083: val_acc did not improve from 1.00000
Epoch 84/100
31/31 [==============================] - 6s 209ms/step - loss: 0.0314 - acc: 0.9960 - val_loss: 0.0827 - val_acc: 0.9820

Epoch 00084: val_acc did not improve from 1.00000
Epoch 85/100
31/31 [==============================] - 6s 187ms/step - loss: 0.0645 - acc: 0.9823 - val_loss: 0.1555 - val_acc: 0.9820

Epoch 00085: val_acc did not improve from 1.00000
Epoch 86/100
31/31 [==============================] - 6s 209ms/step - loss: 0.0704 - acc: 0.9657 - val_loss: 0.0400 - val_acc: 0.9820

Epoch 00086: val_acc did not improve from 1.00000
Epoch 87/100
31/31 [==============================] - 6s 205ms/step - loss: 0.0492 - acc

In [11]:
highest_val_acc, highest_train_acc = max(model_history.history['val_acc']), max(model_history.history['acc'])
print(f'highest test accuracy: {highest_val_acc}')
print('------------------')
print(f'highest train accuracy: {highest_train_acc}')

highest test accuracy: 1.0
------------------
highest train accuracy: 0.9959677419354839


In [12]:
lowest_val_loss, lowest_train_loss = min(model_history.history['val_loss']), min(model_history.history['loss'])
print(f'lowest test loss: {lowest_val_loss}')
print('------------------')
print(f'lowest train loss: {lowest_train_loss}')

lowest test loss: 0.005892951083411505
------------------
lowest train loss: 0.0274533739766765


In [ ]:
plt.plot(model_history.history['val_acc'], color = 'red', label = 'test')
plt.plot(model_history.history['acc'], color = 'blue', label = 'train')
plt.title('accuracy')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(model_history.history['val_loss'], color = 'red', label = 'test')
plt.plot(model_history.history['loss'], color = 'blue', label = 'train')
plt.title('loss')
plt.ylabel('loss')
plt.legend()
plt.show()

### Save model 

In [ ]:
# save model artchitecture 
eyewear_model = model_history.model
eyewear_model_json = eyewear_model.to_json()

In [ ]:
# serialize model to JSON
with open("../tuning_dataeyewear_model.json", "w") as json_file:
    json_file.write(eyewear_model_json)

In [ ]:
# load json and create model
json_file = open('../tuning_data/eyewear_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model_eyewear = model_from_json(loaded_model_json)

In [ ]:
# load model weight 
loaded_model_eyewear.load_weights('../tuning_data/best_vgg_model_eyewears.h5')

In [ ]:
test_gen = ImageDataGenerator(rescale=1./255)
test_generator =  test_gen.flow_from_dataframe(
    dataframe=test_df,
    directory='../data/pics',
    x_col='pic_id',
    y_col='eyewear',
    batch_size=16,
    target_size=(150,150),
    class_mode = 'binary')

In [ ]:
loaded_model_eyewear.compile(loss='binary_crossentropy',
             optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
             metrics=['accuracy'])

In [ ]:
loaded_model_eyewear.evaluate_generator(test_generator)

In [ ]:
feature, label = next(test_generator)

In [ ]:
label[10]

In [ ]:
plt.imshow(feature[10])
plt.show()

In [ ]:
# save the model
np.around(loaded_model_eyewear.predict_generator(test_generator))